# Lesson 03 - v02 - Under and Over Fitting

We will build a VGG-16 model using Keras. This is the same notebook i used for lesson 02, here i will only remove the dropouts for dense layers and make them trainable too, then we will finetune the network and see if the performance increases.

## Build the Model

In [1]:
import numpy as np
import os, json

In [2]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.preprocessing import image
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [3]:
# We will use TensorFlow backend, thus we will explicitly set the Theano image ordering
K.set_image_dim_ordering('th')

 Here i will remove dropouts by setting the p=0. (Dropout(0.)) this will make each node accessible by not removing any of them. In addition we will not need to double the saved dense layer weights, since keras uses inverted dropout which makes the training weights the same as with the case of not using any dropout. https://www.quora.com/What-is-inverted-dropout

In [4]:
# Parameters and helper functions
vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3,1,1))

def vgg_preprocess(x):
    """
        Subtracts the mean RGB value, and transposes RGB to BGR.
        The mean RGB was computed on the image set used to train the VGG model.
        Args: 
            x: Image array (height x width x channels)
        Returns:
        Image array (height x width x transposed_channels)
    """
    x = x - vgg_mean
    return x[:, ::-1] # reverse axis rgb->bgr

def conv_block(n_layers, n_filters):
    """Adds a convolutional block."""
    
    for i in range(n_layers):
        model.add(ZeroPadding2D((1,1)))
        model.add(Convolution2D(n_filters, (3,3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

# Here i will remove dropouts by setting the p=0. (Dropout(0.))
# this will make each node accesable by not removing any of them.
# In addition we will not need to double the saved dense layer weights,
# since keras uses inverse dropout which makes the training weights
# the same as with the case of not using any dropout. 
def fc_block():
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.))

In [5]:
# Build model
model = Sequential()

model.add(Lambda(vgg_preprocess, input_shape=(3,224,224), output_shape=(3,224,224)))
conv_block(2, 64)
conv_block(2, 128)
conv_block(3, 256)
conv_block(3, 512)
conv_block(3, 512)
model.add(Flatten())
fc_block()
fc_block()
model.add(Dense(1000, activation='softmax'))

In [6]:
# Load weights
file_path = 'http://files.fast.ai/models/'
file_name = 'vgg16.h5'
model.load_weights(get_file(file_name, file_path+file_name, cache_subdir='models'))

In [7]:
# load imagenet class index
file_path = 'http://files.fast.ai/models/'
file_name = 'imagenet_class_index.json'
json_path = get_file(file_name, file_path+file_name, cache_subdir='models')
# load json
with open(json_path) as f:
    class_dict = json.load(f)
imagenet_classes = [class_dict[str(i)][1] for i in range(len(class_dict))]

In [8]:
print(len(imagenet_classes))
print(imagenet_classes[:5])

1000
['tench', 'goldfish', 'great_white_shark', 'tiger_shark', 'hammerhead']


In [9]:
class_names = ['cats', 'dogs']

We will remove the final output layer again but in addition we will leave the dense layers (and flattened layer for sure) trainable. Only the conv layers will be un-trainable. 

In [10]:
# get the last conv layer index
last_conv_index = [i for i,layer in enumerate(model.layers) if type(layer) is Convolution2D][-1]

print(last_conv_index)

30


In [11]:
# Remove the last layer and add a layer for cat and dog
# We can remove the last added layer in a Sequential model by calling .pop()
model.pop() # removed the softmax layer
for layer in model.layers[:last_conv_index+1]:
    layer.trainable=False
    #print(type(layer))

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 64, 114, 114)      0         
__________

In [13]:
# we will pop these layers and add back with batch normalizations 
print(type(model.layers[32]))
print(type(model.layers[33]))
print(type(model.layers[35]))

<class 'keras.layers.core.Flatten'>
<class 'keras.layers.core.Dense'>
<class 'keras.layers.core.Dense'>


In [14]:
dense_layers = [model.layers[32], model.layers[33], model.layers[35]]

for i in range(5):
    model.pop()

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 64, 114, 114)      0         
__________

In [16]:
model.add(dense_layers[0])
model.add(BatchNormalization())
model.add(dense_layers[1])
model.add(BatchNormalization())
model.add(dense_layers[2])
model.add(BatchNormalization())
# add final layer
model.add(Dense(2, activation='softmax'))

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 64, 114, 114)      0         
__________

Here we will use RMSprop and since we are really **fine** tuning the dense layers, we will choose a very small learning rate.

In [18]:
# Compile the model
model.compile(
    optimizer=RMSprop(lr=0.000001, rho=0.7), loss='categorical_crossentropy', metrics=['accuracy'])

## Train

In [19]:
batch_size = 64

# Generators
gen = image.ImageDataGenerator()

train_batches = gen.flow_from_directory(
    'data/train-fast-ai',
    target_size=(224,224),
    class_mode='categorical',
    shuffle=True,
    batch_size=batch_size)

valid_batches = gen.flow_from_directory(
    'data/valid-fast-ai',
    target_size=(224,224),
    class_mode='categorical',
    shuffle=False,
    batch_size=batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [20]:
print(train_batches.samples)
print(valid_batches.samples)

23000
2000


In [21]:
# Train
model.fit_generator(
    train_batches, steps_per_epoch=train_batches.samples//batch_size,
    epochs=7, validation_data=valid_batches,
    validation_steps=valid_batches.samples//batch_size)

Epoch 1/7
359/359 [==============================] - 130s - loss: 0.3226 - acc: 0.8587 - val_loss: 0.1874 - val_acc: 0.9254
Epoch 2/7
359/359 [==============================] - 129s - loss: 0.1576 - acc: 0.9379 - val_loss: 0.1493 - val_acc: 0.9411
Epoch 3/7
359/359 [==============================] - 127s - loss: 0.1191 - acc: 0.9548 - val_loss: 0.1332 - val_acc: 0.9468
Epoch 4/7
359/359 [==============================] - 127s - loss: 0.0921 - acc: 0.9673 - val_loss: 0.1189 - val_acc: 0.9489
Epoch 5/7
359/359 [==============================] - 131s - loss: 0.0685 - acc: 0.9781 - val_loss: 0.1210 - val_acc: 0.9489
Epoch 6/7
359/359 [==============================] - 134s - loss: 0.0549 - acc: 0.9829 - val_loss: 0.1168 - val_acc: 0.9494
Epoch 7/7
359/359 [==============================] - 135s - loss: 0.0428 - acc: 0.9874 - val_loss: 0.1172 - val_acc: 0.9525


With batch normalization, loss decreases consistently also, even the val accuracy is lower than the previous one, this predictions scored better on kaggle.

In [22]:
# save weights
%mkdir -p saved
model.save_weights('saved/saved_weights_wo_dropout_w_batchnorm.h5')

## Predict and submit

In [23]:
batch_size = 25
gen = image.ImageDataGenerator()

test_batches = gen.flow_from_directory(
    './data/test',
    target_size=(224,224),
    class_mode=None,
    shuffle=False,
    batch_size=batch_size)

Found 12500 images belonging to 1 classes.


In [24]:
preds = model.predict_generator(test_batches, test_batches.samples//batch_size)

In [25]:
from importlib import reload
import utils
reload(utils)
from utils import submit2redux

In [26]:
submit2redux(test_batches, preds, file_name='subm_wo_dropout_w_batchnorm.csv')

File saved as ./data/subm_wo_dropout_w_batchnorm.csv.
